# Atomic

## arch/x86/include/asm/barrier.h

```c
/*
 * Force strict CPU ordering.
 * And yes, this is required on UP too when we're talking
 * to devices.
 */

#ifdef CONFIG_X86_32
/*
 * Some non-Intel clones support out of order store. wmb() ceases to be a
 * nop for these.
 */
#define mb() alternative("lock; addl $0,0(%%esp)", "mfence", X86_FEATURE_XMM2)
#define rmb() alternative("lock; addl $0,0(%%esp)", "lfence", X86_FEATURE_XMM2)
#define wmb() alternative("lock; addl $0,0(%%esp)", "sfence", X86_FEATURE_XMM)
#else
#define mb() 	asm volatile("mfence":::"memory")
#define rmb()	asm volatile("lfence":::"memory")
#define wmb()	asm volatile("sfence" ::: "memory")
#endif
```

1. If cpu doesn't support `mfence,lfence,sfence`, use `lock` instruction. The LOCK prefix ensures that the CPU has exclusive ownership of the appropriate cache line for the duration of the operation, and provides certain additional ordering guarantees. This may be achieved by asserting a bus lock, but the CPU will avoid this where possible. 

2. [Lock & memory order](https://stackoverflow.com/questions/60332591/why-is-lock-a-full-barrier-on-x86)

3. [缓存一致性MESI](https://cloud.tencent.com/developer/article/1548942)

MESI优化和他们引入的问题
缓存的一致性消息传递是要时间的，这就使其切换时会产生延迟。当一个缓存被切换状态时其他缓存收到消息完成各自的切换并且发出回应消息这么一长串的时间中CPU都会等待所有缓存响应完成。可能出现的阻塞都会导致各种各样的性能问题和稳定性问题。

CPU切换状态阻塞解决-存储缓存（Store Bufferes）

比如你需要修改本地缓存中的一条信息，那么你必须将I（无效）状态通知到其他拥有该缓存数据的CPU缓存中，并且等待确认。等待确认的过程会阻塞处理器，这会降低处理器的性能。应为这个等待远远比一个指令的执行时间长的多。

Store Bufferes

为了避免这种CPU运算能力的浪费，Store Bufferes被引入使用。处理器把它想要写入到主存的值写到缓存，然后继续去处理其他事情。当所有失效确认（Invalidate Acknowledge）都接收到时，数据才会最终被提交。

这么做有两个风险

Store Bufferes的风险

第一、就是处理器会尝试从存储缓存（Store buffer）中读取值，但它还没有进行提交。这个的解决方案称为Store Forwarding，它使得加载的时候，如果存储缓存中存在，则进行返回。

第二、保存什么时候会完成，这个并没有任何保证。

**写屏障 Store Memory Barrier(a.k.a. ST, SMB, smp_wmb)是一条告诉处理器在执行这之后的指令之前，应用所有已经在存储缓存（store buffer）中的保存的指令。**

**读屏障Load Memory Barrier (a.k.a. LD, RMB, smp_rmb)是一条告诉处理器在执行任何的加载前，先应用所有已经在失效队列中的失效操作的指令。**

**这正是为什么 release和acquire要成对出现的原因。**


4. Compiler Barriers

对编译器的优化我们可以使用compiler barrier，比如大家熟知的"volatile"，就可以让编译器生成的代码，每次都从内存重新读取变量的值，而不是用寄存器中暂存的值。因为在多线程环境中，不会被当前线程修改的变量，可能会被其他的线程修改，从内存读才可靠。

这就部分解释了上文留的那个问题，即为什么要用READ_ONCE()和WRITE_ONCE()这两个宏，因为atomic_read()和atomic_set()所操作的这个变量，可能会被多核/多线程同时修改，需要避免编译器把它当成一个普通的变量，做出错误的优化。还有一部分原因是，这两个宏可以作为标记，提醒编程人员这里面是一个多核/多线程共享的变量，必要的时候应该加互斥锁来保护。

Linux中设置compiler barrier的函数是barrier()，它对应gcc的实现是这样的（定义在include/linux/compiler-gcc.h）：

/* The "volatile" is due to gcc bugs */
#define barrier() __asm__ __volatile__("": : :"memory")
这是一个内嵌汇编，里是一个空的指令，空的指令怎么发挥作用？

它其实利用了末尾clobber list里的"memory"，clober list是gcc和gas(GNU Assembler)的接口，用于gas通知gcc它对寄存器和memory的修改情况。

这里的"memory"就是告知gcc，在汇编代码中，我修改了内存中的内容，之前的C代码块和之后的C代码块看到的内存是不一样的，对内存的访问不能依赖于嵌入汇编之前的C代码块中寄存器的内容，所以乖乖地重新从内存读数据吧。

也不知道编译器能不能识别这种伎俩，反正最后它是欣然的被骗了。需要注意的是，barrier()只会对编译器的行为产生约束，它不会生成真正的指令，因此对最终CPU的指令执行没有影响。

Linux还提供了一个函数叫smp_mb()，看起来好像是专门用于SMP系统的memory barrier，那它能提供SMP系统中，不同CPU对内存访问顺序的保证吗？不能，SMP系统中，它就等同于mb()，在UP系统中，它会退化为compiler barrier。

----------------------------

```c
#ifdef CONFIG_SMP
#define smp_mb()	mb()
#else	
#define smp_mb()	barrier()
#endif
```

smp_mb()并不像很多人理解的那样，是mb()的超集(superset)，相反，它只能算mb()的子集(subset)。能用smp_mb()的地方可以用mb()代替，但能用mb()的地方不一定能用smp_mb()代替。

即使在多核系统上smp_mb也比mb弱。smp_mb的作用范围局限在CPU cores之间，mb的作用范围包括CPU cores和SoC上其他模块。

5. [linux barrier](https://zhuanlan.zhihu.com/p/96001570)

6. [barrier and linux kernel](https://community.arm.com/arm-community-blogs/b/architectures-and-processors-blog/posts/memory-access-ordering-part-2---barriers-and-the-linux-kernel)

----------------------------

```c
/* Optimization barrier */
/* The "volatile" is due to gcc bugs */
#define barrier() __asm__ __volatile__("": : :"memory")
```

1. this is the barrier definition. It's a compiler barrier.

`volatile` and `memory` both tell the compiler read value from memory not from registers. All memory access operations before the barrier can't be reordered behind it and same as the operations behind it.

----------------------------------

```c
#ifdef CONFIG_X86_PPRO_FENCE
#define dma_rmb()	rmb()
#else
#define dma_rmb()	barrier()
#endif
#define dma_wmb()	barrier()

#ifdef CONFIG_SMP
#define smp_mb()	mb()
#define smp_rmb()	dma_rmb()
#define smp_wmb()	barrier()
#define smp_store_mb(var, value) do { (void)xchg(&var, value); } while (0)
#else /* !SMP */
#define smp_mb()	barrier()
#define smp_rmb()	barrier()
#define smp_wmb()	barrier()
#define smp_store_mb(var, value) do { WRITE_ONCE(var, value); barrier(); } while (0)
#endif /* SMP */
```

1. `barrier()` is just a compiler barrier. For UP, compiler barrier is enougth. 

2. for SMP, `smp_wmb` only need compiler barrier, because for x86, 而在x86中，对于同一CPU执行的load指令后接load指令（L-L），store指令后接store指令（S-S），load指令后接store指令（L-S），都是不能交换指令的执行顺序的，只有store指令后接load指令（S-L）时才可以[注1]。这种memory order被称为TSO(Total Store Order)，俗称strong order。

也就是说，`write` 在cpu中不会重排到其他write前面，既然cpu不这么做，我们只要约束compiler不重排就好了

3. 对于 rmb和mb，就要用`mfence` 和 `lfence` 来约束cpu的行为

-------------------------